<a href="https://colab.research.google.com/github/bytebuster21/AI-project/blob/main/Adding_Dense_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16 # Import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
import matplotlib.pyplot as plt
import os
import shutil

from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

# --- 2. Define your dataset paths ---
trainpath = '/content/train'
testpath = '/content/test'

if not os.path.exists(trainpath):
    print("Creating dummy dataset for demonstration...")
    os.makedirs(os.path.join(trainpath, 'class_A'), exist_ok=True)
    os.makedirs(os.path.join(trainpath, 'class_B'), exist_ok=True)
    os.makedirs(os.path.join(trainpath, 'class_C'), exist_ok=True)

    os.makedirs(os.path.join(testpath, 'class_A'), exist_ok=True)
    os.makedirs(os.path.join(testpath, 'class_B'), exist_ok=True)
    os.makedirs(os.path.join(testpath, 'class_C'), exist_ok=True)

    from PIL import Image
    def create_dummy_image(path, color, size=(299, 299)):
        img = Image.new('RGB', size, color=color)
        img.save(path)

    # Create dummy training images
    for i in range(116): # 116 * 3 classes = 348
        create_dummy_image(os.path.join(trainpath, 'class_A', f'imgA_{i:03d}.png'), (255, 100, 100))
        create_dummy_image(os.path.join(trainpath, 'class_B', f'imgB_{i:03d}.png'), (100, 255, 100))
        create_dummy_image(os.path.join(trainpath, 'class_C', f'imgC_{i:03d}.png'), (100, 100, 255))

    # Create dummy testing images
    for i in range(51): # 51 * 3 classes = 153
        create_dummy_image(os.path.join(testpath, 'class_A', f'testA_{i:03d}.png'), (200, 50, 50))
        create_dummy_image(os.path.join(testpath, 'class_B', f'testB_{i:03d}.png'), (50, 200, 50))
        create_dummy_image(os.path.join(testpath, 'class_C', f'testC_{i:03d}.png'), (50, 50, 200))

    print(f"Dummy dataset created at {trainpath} and {testpath}")
else:
    print(f"Using existing dataset at {trainpath} and {testpath}")


# --- 4. Configure ImageDataGenerator instances ---
# VGG16 can accept various sizes, 299x299 is specified for this setup
TARGET_SIZE = (299, 299)
BATCH_SIZE = 20

# Training Data Generator (with augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Test Data Generator (ONLY rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# --- 5. Flow Images from Directories ---
print("\nLoading images using ImageDataGenerator...")

train_generator = train_datagen.flow_from_directory(
    trainpath,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    testpath,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Get class names and number of classes from the generator
num_classes = train_generator.num_classes
class_names = list(train_generator.class_indices.keys())

print(f"Found {train_generator.samples} training images belonging to {num_classes} classes.")
print(f"Found {test_generator.samples} test images belonging to {num_classes} classes.")
print("Class Names:", class_names)


# --- 6. Build the Model using VGG16 as a Pre-trained Base (Feature Extractor) and Add Custom Layers ---

# Load the VGG16 model with pre-trained ImageNet weights.
# `include_top=False` means we are not including the ImageNet classification layers
# at the top of the network, as we will add our own for our specific classification task.
base_model = VGG16(
    weights='imagenet',        # Use weights pre-trained on ImageNet
    include_top=False,         # Exclude the classifier layers
    input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3) # Define the input image shape
)

# Freeze the layers of the base model.
# This is crucial to use VGG16 as a fixed feature extractor.
# It ensures that the pre-trained weights of all five convolution blocks
# don’t get updated during the training of the new custom layers.
base_model.trainable = False

# Create a new model by adding custom Dense layers on top of the VGG16 base.
model = Sequential([
    base_model, # Add the pre-trained VGG16 convolutional base
    Flatten(),  # Flatten the output from the convolutional layers into a 1D vector.
                # This layer does not affect the batch size; it only reshapes the feature map.
    Dense(256, activation='relu'), # First custom Dense layer to learn high-level features
    Dropout(0.5), # Dropout for regularization to prevent overfitting
    Dense(128, activation='relu'), # Second custom Dense layer for further feature learning
    Dropout(0.4), # Second Dropout layer
    Dense(num_classes, activation='softmax') # Output layer with 'softmax' for multi-class classification
])

# Compile the model.
# Use the Adam optimizer and categorical_crossentropy loss for multi-class classification.
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the summary of the complete model, including the base and custom layers.
print("\n--- Complete Model Summary (VGG16 with Custom Dense Layers) ---")
model.summary()

# --- 7. Train the Model (Uncomment to run) ---
print("\n--- Starting Model Training (Commented out - uncomment to run) ---")



Mounted at /content/drive
Google Drive mounted successfully!
Creating dummy dataset for demonstration...
Dummy dataset created at /content/train and /content/test

Loading images using ImageDataGenerator...
Found 348 images belonging to 3 classes.
Found 153 images belonging to 3 classes.
Found 348 training images belonging to 3 classes.
Found 153 test images belonging to 3 classes.
Class Names: ['class_A', 'class_B', 'class_C']
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

--- Complete Model Summary (VGG16 with Custom Dense Layers) ---


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 9, 9, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    10,617,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,365,059 (96.76 MB)

 Trainable params: 10,650,371 (40.63 MB)

 Non-trainable params: 14,714,688 (56.13 MB)


--- Starting Model Training (Commented out - uncomment to run) ---
